## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-22-53-49 +0000,nypost,Netanyahu believes occupying Gaza is the only ...,https://nypost.com/2025/08/05/world-news/netan...
1,2025-08-05-22-51-00 +0000,wsj,The Trump administration ran afoul of the Impo...,https://www.wsj.com/politics/policy/nih-fundin...
2,2025-08-05-22-48-00 +0000,wsj,"News Corp Earnings Increase, Lifted by Dow Jon...",https://www.wsj.com/business/media/news-corp-e...
3,2025-08-05-22-42-14 +0000,nypost,Day care worker arrested after allegedly forci...,https://nypost.com/2025/08/05/us-news/day-care...
4,2025-08-05-22-41-24 +0000,nyt,Edinburgh Fringe in Photos: The Streets Are Alive,https://www.nytimes.com/2025/08/04/theater/edi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,56
63,house,15
124,new,14
147,out,13
13,war,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
296,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,98
15,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,96
92,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...,95
94,2025-08-05-19-37-53 +0000,nyt,"Ahead of Trump’s Trade Deadline, Firms Rushed ...",https://www.nytimes.com/2025/08/01/business/ta...,94
316,2025-08-04-22-55-43 +0000,latimes,Trump plans White House task force on security...,https://www.latimes.com/politics/story/2025-08...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
296,98,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...
0,57,2025-08-05-22-53-49 +0000,nypost,Netanyahu believes occupying Gaza is the only ...,https://nypost.com/2025/08/05/world-news/netan...
207,52,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
283,41,2025-08-05-02-03-35 +0000,nyt,"In Delhi and New York, Hindu Right Wing Lines ...",https://www.nytimes.com/2025/08/01/world/asia/...
60,40,2025-08-05-20-49-43 +0000,nypost,Ghislaine Maxwell rejects DOJ bid to unseal gr...,https://nypost.com/2025/08/05/us-news/ghislain...
291,38,2025-08-05-01-12-00 +0000,wsj,Brazil’s Supreme Court Orders Ex-President Bol...,https://www.wsj.com/world/americas/brazils-sup...
43,37,2025-08-05-21-17-37 +0000,nypost,Eric Adams’ re-election campaign gets boost fr...,https://nypost.com/2025/08/05/us-news/eric-ada...
209,36,2025-08-05-13-27-00 +0000,cbc,"India, Russia hit back after U.S. accuses Modi...",https://www.cbc.ca/news/world/india-us-russia-...
290,35,2025-08-05-01-17-08 +0000,latimes,Wealthy first-time candidate Cloobeck drops $1...,https://www.latimes.com/california/story/2025-...
73,35,2025-08-05-20-27-15 +0000,nypost,Tennessee death row inmate cries out in pain d...,https://nypost.com/2025/08/05/us-news/tennesse...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
